# Модель для генерации отзывов о ресторане

# Препроцессинг и обучение модели

## Загрузка данных и подготовка среды для работы

In [15]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling, TextDataset
from huggingface_hub import HfApi
from transformers import pipeline

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
MODEL_OUT_DIR= "dialogue_reviews"
FILE_PATH = 'reviews.txt'
MODEL_CKPT = 'gpt2'
MODEL_NAME = "yana-sklyanchuk/dialogue_reviews"  # your model path
push_to_hub = True

In [18]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_CKPT)
model = GPT2LMHeadModel.from_pretrained(MODEL_CKPT)

# Load your dataset
def load_dataset(FILE_PATH):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=FILE_PATH,
        block_size=128  # Adjust this based on your needs
    )

# Create a text dataset from reviews.txt
train_dataset = load_dataset(FILE_PATH)

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up training arguments
training_args = TrainingArguments(
    output_dir=MODEL_OUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    push_to_hub=PUSH_TO_HUB
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Push the model and tokenizer to Hugging Face Hub
if push_to_hub:
    trainer.push_to_hub()
    tokenizer.push_to_hub('yana-sklyanchuk/dialogue_reviews')

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


README.md:   0%|          | 0.00/993 [00:00<?, ?B/s]

## Проверка работы модели

In [19]:

# Load the fine-tuned model and tokenizer from the Hugging Face Hub
generator = pipeline('text-generation', model=MODEL_NAME)

# Define a prompt for text generation
prompt = "The restaurant had"

# Generate text based on the prompt
generated_texts = generator(prompt, max_length=50, num_return_sequences=3)

# Print the generated texts
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}: {text['generated_text']}")

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: The restaurant had a very nice service and had good prices for dessert.
The only complaint I have is the fish I have was not cooked well.
If you want a good sushi takeout, this would be your location.
When I was
Generated Text 2: The restaurant had a fabulous vibe.
At least it was tastier with fries, not the shrimp.
I've been craving fried meat ever since the last time I tried this place.
The staff is friendly and friendly and it's not like
Generated Text 3: The restaurant had wonderful service, and the staff and staff were great.
Best burgers of 2015!
Went here for lunch...just not really a good lunch.
I went here to get your "The Best Steak in Vegas".



In [31]:
# Generate text based on the prompt
generated_texts = generator(prompt, max_length=16, num_return_sequences=1)

# Print the generated texts
for i, text in enumerate(generated_texts):
    print(text['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The restaurant had a lovely setting which was a great surprise to watch.
There


# Выводы

Эта модель представляет собой доработанную версию gpt2 для создания отзывов о ресторане.



##Процедура обучения


### Гиперпараметры обучения


Во время обучения использовались следующие гиперпараметры:

- learning_rate: 5e-05
- train_batch_size: 2
- eval_batch_size: 8
- seed: 42
- optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
- lr_scheduler_type: linear
- num_epochs: 3


## Framework versions

- Transformers 4.42.4
- Pytorch 2.3.1+cu121
- Tokenizers 0.19.1